In [1]:
!pip install efficientnet_pytorch torch torchvision

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16429 sha256=672168f80dbdfc9cd47332aaff20e19d7bbf5af304093893ba89bda638c78971
  Stored in directory: /home/ec2-user/.cache/pip/wheels/22/16/f1/5369d23a06852d5f083d23a1addf0904575f1296f71b412ac8
Successfully built efficientnet_pytorch


# Data Processsing

In [2]:
train_dir = '/home/ec2-user/SageMaker/train/'
test_dir = '/home/ec2-user/SageMaker/test/'
files_dir = '/home/ec2-user/SageMaker/'

## Labels

In [3]:
import json

with open(files_dir + '/label_num_to_disease_map.json', 'r') as f:
    labels_info = json.load(f)

NUM_LABELS = len(labels_info)
print(labels_info)

{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}


In [4]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv(files_dir + '/train.csv')

# Split the data into train, validation, and test sets with a ratio of 80:10:10
train_df, val_test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

In [6]:
class CassavaDataset(Dataset):
    def __init__(self, data, root_dir, transform=None):
        self.image_labels = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.image_labels.iloc[index, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.image_labels.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

## Useful Function

In [7]:
# Save the model so that we don't need to train the same model multiple times
def save_model(model, model_name):
  torch.save(model.state_dict(),  files_dir + 'saved_models/' + model_name + '.pth')

# Training with Efficient Net

In [8]:
# parameters
BATCH_SIZE = 32
EPOCHS = 5
IMG_SIZE = 224
LR = 1e-4

In [9]:
# Load the Cassava Leaf Disease dataset (assuming it's in 'cassava_dataset' directory)
data_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_dataset = CassavaDataset(train_df, train_dir, transform=data_transforms)
validation_dataset = CassavaDataset(val_df, train_dir, transform=data_transforms)
test_dataset = CassavaDataset(test_df, train_dir, transform=data_transforms)

In [10]:
# Create the data loader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [21]:
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from efficientnet_pytorch import EfficientNet


# Load the pre-trained EfficientNet model
model = EfficientNet.from_pretrained('efficientnet-b3')

# Linear Layers
model._fc = torch.nn.Sequential(
    torch.nn.Linear(model._fc.in_features, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, NUM_LABELS)
)

# Set up the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# Fine-tune the EfficientNet model
num_epochs = EPOCHS
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
  

### Start training

In [22]:

train_loss, validation_loss = [], []
train_acc, validation_acc = [], []

for epoch in range(EPOCHS):
    print("Training Epoch: ", epoch)
    batch_num = 0
    
    model.train()
    running_loss = 0.
    correct, total = 0, 0 
    total_acc= 0.0
    for inputs, labels in train_loader:
        batch_num += 1
        if batch_num % 50 == 0:
            print("Current Batch: ", batch_num)
        # Forward pass
        inputs = inputs.to(device)
        labels = labels.to(device)
         
        predictions = model(inputs)
        loss = criterion(predictions, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = (predictions.argmax(dim=1) == labels).float().mean()
        total_acc += acc
        
        running_loss += loss.item()
            
        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss.append(running_loss / len(train_loader))
    train_acc.append(correct/total)
    print("Train Accuracy:", total_acc / total)

    model.eval()
    running_loss = 0.
    correct, total = 0, 0 
    
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        predictions = model(inputs)

        loss = criterion(predictions, labels)

        running_loss += loss.item()

        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    validation_loss.append(running_loss / len(val_loader))
    validation_acc.append(correct/total)
    
    print(f"Epoch {epoch+1}:")

    print(f"Training Loss:", round(train_loss[epoch], 3))
    print(f"Validation Loss:", round(validation_loss[epoch], 3))

    print(f"Training Accuracy:", round(train_acc[epoch], 3))
    print(f"Validation Accuracy:", round(validation_acc[epoch], 3))

    print("------------------------------")

Training Epoch:  0
Current Batch:  50
Current Batch:  100
Current Batch:  150
Current Batch:  200
Current Batch:  250
Current Batch:  300
Current Batch:  350
Current Batch:  400
Current Batch:  450
Current Batch:  500
Train Accuracy: tensor(0.0231, device='cuda:0')
Epoch 1:
Training Loss: 0.728
Validation Loss: 0.518
Training Accuracy: 0.738
Validation Accuracy: 0.823
------------------------------
Training Epoch:  1
Current Batch:  50
Current Batch:  100
Current Batch:  150
Current Batch:  200
Current Batch:  250
Current Batch:  300
Current Batch:  350
Current Batch:  400
Current Batch:  450
Current Batch:  500
Train Accuracy: tensor(0.0266, device='cuda:0')
Epoch 2:
Training Loss: 0.414
Validation Loss: 0.521
Training Accuracy: 0.851
Validation Accuracy: 0.819
------------------------------
Training Epoch:  2
Current Batch:  50
Current Batch:  100
Current Batch:  150
Current Batch:  200
Current Batch:  250
Current Batch:  300
Current Batch:  350
Current Batch:  400
Current Batch:  45

In [23]:
save_model(model, "effNet_b3_5_epochs_32_batch")

### Model Evaluation

In [25]:
import numpy as np

model.eval()

test_predictions = np.array([])
true_labels = np.array([])

total = 0
correct = 0

for inputs, labels in test_loader:
    
    inputs = inputs.to(device)
    labels = labels.to(device)

    predictions = model(inputs)
    
    _, predicted = torch.max(predictions, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
    true_labels = np.append(true_labels, labels.cpu().numpy())
    test_predictions = np.concatenate((test_predictions, predicted.detach().cpu().numpy()))
    
print("Test Accuracy: ", correct/total)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.78 GiB total capacity; 14.06 GiB already allocated; 4.94 MiB free; 14.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

f1 = f1_score(true_labels, test_predictions, average='weighted')
recall = recall_score(true_labels, test_predictions, average='weighted')
precision = precision_score(true_labels, test_predictions, average='weighted')
accuracy = accuracy_score(true_labels, test_predictions)

# Print the accuracy, F1 score, recall, and precision
print("Test Accuracy: ", accuracy)
print("F1 Score: ", f1)
print("Recall: ", recall)
print("Precision: ", precision)